In [4]:
import pickle
import pandas as pd

### Compound names from MeSh substance headings


In [5]:
mesh_compounds = ['dihydropyridines?', 'benzothiazepines?', 'phenyl[- ]?alkyl[- ]?amines?', 'AE0047' , 'Amlodipine' , 'Amrinone' , 'anandamide' , 'anipamil' , 'azimilide' , 'Bencyclane' , 'benidipine' , 'Bepridil' , 'berbamine' , 'canadine' , 'carboxyamido-triazoles?' , 'caroverine' , 'CD 349' , 'cilnidipine' , 'Cinnarizine' , 'clentiazem' , 'clevidipine' , 'Conotoxins' , 'Conus magus toxin' , 'darodipine' , 'dauricine' , 'devapamil' , 'Diltiazem' , 'dotarizine' , 'efonidipine' , 'emopamil' , 'enpiperate' , 'eperisone' , 'falipamil' , 'fantofarone' , 'fasudil' , 'Felodipine' , 'fenamic acid' , 'Fendiline' , 'Flunarizine' , 'fosfedil' , 'FTX' , 'gabapentin' , 'Gallopamil' , 'IH 764-3' , 'Isradipine' , 'KT 362' , 'lacidipine' , 'lamotrigine' , 'lercanidipine' , 'Lidoflazine' , 'lomerizine' , 'manidipine' , 'manoalide' , 'mepirodipine' , 'Mibefradil' , 'monatepil' , 'N-(2-guanidinoethyl)-5-isoquinolinesulfonamide' , 'naftopidil' , 'Nicardipine' , 'Nifedipine' , 'niguldipine' , 'niludipin' , 'nilvadipine' , 'Nimodipine' , 'Nisoldipine' , 'Nitrendipine' , 'norverapamil' , 'ochratoxin A' , 'octylonium' , 'Agatoxins?' , 'osthol' , 'oxodipine' , 'Perhexiline' , 'pinaverium' , 'piperidines?' , 'PN 202-791' , 'pranidipine' , 'pregabalin' , 'Prenylamine' , 'R 56865' , 'risedronic acid' , 'ryodipine' , 'SAN 202791' , 'sesamodil' , 'stepholidine' , 'temiverine' , 'terodiline' , 'tetrahydropalmatine' , 'tetrandrine' , 'Tiapamil' , 'tolfenamic acid' , 'tranilast' , 'Verapamil' , 'Win 55212-2' , 'ziconotide']
mesh_compounds.remove('N-(2-guanidinoethyl)-5-isoquinolinesulfonamide')

In [6]:
len(mesh_compounds)

94

In [7]:
mesh_compounds.append('N-\(2-guanidinoethyl\)-5-isoquinolinesulfonamide')

In [8]:
mesh_compounds = [i.lower() for i in mesh_compounds]

### Preparing Wisniowska compounds to add to the list above


In [9]:
Wisniowska_suppl = pd.read_csv('Wisniowska_suppl.txt', sep = '\t')

In [10]:
master_list = [i.lower() for i in Wisniowska_suppl['Compound'].unique()[2:]]

In [11]:
len(master_list)

69

In [12]:
master_list.extend(mesh_compounds)

In [13]:
len(master_list)

164

In [14]:
master_list = list(set(master_list))

In [15]:
len(master_list)

148

In [16]:
master_list.remove('sr33557 (fantofarone)')

In [17]:
master_list.append('sr33557')

In [18]:
master_list.append('fantofarone')

In [19]:
master_list.remove('vanoredxine (gbr-12909)')

In [20]:
master_list.append('vanoredxine')

In [21]:
master_list.append('gbr-12909')

In [22]:
len(master_list)

150

### Define highlighting dictionaries

In [56]:
broader_terms = [
    {
          'name': 'chem'
        , 'terms': ['compounds?', 'molecules?', 'analog\w*', 'ligands?', 'binding', 'potency', 'potencies', 'affinit(?:y|ies)', 'displacement', 'displacing', 'chemicals?', ]
        , 'color': 'LightSeaGreen'
    }
    , {
          'name':'IC50s'
         ,'terms': ['[IEXA][CD][0-9]0s?', '[IEXA][CD]\([0-9]0\)s?', 'inhibitory activit(?:y|ies)']
         ,'color': Goldenrod}
    , {
          'name': 'calcium'
        , 'terms': ['calcium channel?','calcium','targets?', '(calcium|ca) (channels?|currents?)', 'channels?', 'L-type',
                    'Ca2?(\+)+[- ]channels?', 'L-type (calcium|ca)[ -](channels?|currents?)', 'L-type Ca2?(\+)+', 'L-type Ca2?(\+)+[- ](channels?|currents?)', 'LTCCs?',
                   'ICa, ?L', 'I\(Ca,L\)', 'DHPRs?', 'Cav[0-9]\.[0-9a-z]+', 'DHP receptors?', 'DHP channels?', 'calcium channels?', 'voltage-dependent (?:ca|calcium)',
                    'voltage-gated (?:ca|calcium)', 'Ca\(2\+\)[- ]?channel[- s]', 'Ca\(2\+\)[ -]channels?',
                     'calcium[- ]entry']
        , 'color': 'blue'
    }
    , {
          'name': 'other_terms'
        , 'terms': ['(?:modulator|antagonis|inhibitor|block)\w*']
        , 'color': 'RoyalBlue'
    }
    ,{
        'name':'interesting_tissue'
        , 'terms': ['cardiac', 'heart', 'cardio[\w-]+', 'atri(?:al|um)', 'ventricles?', 'ventricular', 'myocardi(?:al|um)', 'papillary']
        , 'color': 'Magenta'
    }
    , {
          'name':'unwanted_tissue'
        , 'terms':['ileal','ileum', 'smooth muscle', 'vascular muscle', 'nervous', 'CNS', 'brain', 'contractions?', 'neurons?', 'skeletal muscle', 'vasodilatory', 'vascular', 'vaso-?(?:dilat|constrict|relax)\w+', 'constriction', 'dilator\w*', 'bronchiol\w*', 'aorta']
        , 'color':'red'
    } 
    , {
        'name':'effect'
        , 'terms': ['inotrop\w+', 'lusitrop\w+', 'dromotrop\w+', 'tachycard\w+', 'bradycard\w+', '(?:anti[- ]?)?arrhythmi\w+', 'tdp', 'torsade de pointes', 'QT', 'phasic', 'tonic', 'voltage[- ](?:sensitive|dependent)', 'use[- ]dependent']
        , 'color': 'RosyBrown'
    }
    , {
        'name':'sodium'
        , 'terms': ['sodium', 'Na']
        , 'color': 'orange'
    }
    , {
        'name':'assays'
        , 'terms': ['(?:patch|voltage)[- ]clamp\w*', 'IonWorks', 'electrophysiolog\w*', 'patchxpress']
        , 'color': 'Chocolate'
    }
    , {
        'name':'species'
        , 'terms': ['guinea[- ]pig', 'rabbit', 'rat', 'mouse', 'dog', 'human', 'pig']
        , 'color': 'MediumOrchid'
    },
    {
        'name': 'chemicals'
        ,'terms': master_list
        ,'color':'MediumSeaGreen'
    }
]

NameError: name 'Goldenrod' is not defined

In [57]:
with open('terms_for_highlighting.pkl', 'wb') as f:
    
    pickle.dump(broader_terms, f)     